In [1]:
# Como siempre, primero nos importamos las librerías con las que vamos a estar trabajando

import requests # Nos va a permitir hacer llamados con el protocolo HTTP
import json # Nos va a permitir trabajar con los archivos JSON
import pandas as pd # Nuestra librería amiga de siempre que nos permite trabajar con datos tabulares
import numpy as np #numpy para trabajar con operaciones matemáticas y arrays
import urllib.request
from PIL import Image
import os

In [2]:
# Nos definimos una función que nos va a dejar acceder a la información que nos devuelva el URL que pongamos
# en el argumento. Recuerden que el URL es la dirección del sitio web que vamos a usar

def get_response(url):
    try:
        response = requests.get(url, timeout=3)
        response.raise_for_status()
        return response
    except requests.exceptions.HTTPError as errh:
        print ("Http Error:",errh)
    except requests.exceptions.ConnectionError as errc:
        print ("Error Connecting:",errc)
    except requests.exceptions.Timeout as errt:
        print ("Timeout Error:",errt)
    except requests.exceptions.RequestException as err:
        print ("OOps: Something Else",err)

In [3]:
# Definamos que categorías principales hay en el sitio
url='https://api.mercadolibre.com/sites/MLA/categories'
categorias_full=requests.get(url).json()

In [4]:
categorias_full

[{'id': 'MLA5725', 'name': 'Accesorios para Vehículos'},
 {'id': 'MLA1512', 'name': 'Agro'},
 {'id': 'MLA1403', 'name': 'Alimentos y Bebidas'},
 {'id': 'MLA1071', 'name': 'Animales y Mascotas'},
 {'id': 'MLA1367', 'name': 'Antigüedades y Colecciones'},
 {'id': 'MLA1368', 'name': 'Arte, Librería y Mercería'},
 {'id': 'MLA1743', 'name': 'Autos, Motos y Otros'},
 {'id': 'MLA1384', 'name': 'Bebés'},
 {'id': 'MLA1246', 'name': 'Belleza y Cuidado Personal'},
 {'id': 'MLA1039', 'name': 'Cámaras y Accesorios'},
 {'id': 'MLA1051', 'name': 'Celulares y Teléfonos'},
 {'id': 'MLA1648', 'name': 'Computación'},
 {'id': 'MLA1144', 'name': 'Consolas y Videojuegos'},
 {'id': 'MLA1500', 'name': 'Construcción'},
 {'id': 'MLA1276', 'name': 'Deportes y Fitness'},
 {'id': 'MLA5726', 'name': 'Electrodomésticos y Aires Ac.'},
 {'id': 'MLA1000', 'name': 'Electrónica, Audio y Video'},
 {'id': 'MLA2547', 'name': 'Entradas para Eventos'},
 {'id': 'MLA407134', 'name': 'Herramientas'},
 {'id': 'MLA1574', 'name': 'H

In [5]:
# Definimos una función para traer los items de una búsqueda cambiando la categoria
def buscaritems(categ,offset=0,maximum=None):
    # Primero indicamos la consulta que vamos a hacer
    url='https://api.mercadolibre.com/sites/MLA/search?category={}&limit=50&offset=0#json'.format(categ)
    r = get_response(url) #Vemos los resultados de la consulta
    if maximum==None:
        #Consultamos cuál es el número máximo de resultados
        maximum = int(str(r.json()["paging"]["total"]))
    print('Encontramos {} resultados para nuestra consulta'.format(maximum))
    # Vamos a forzarnos a buscar pocos resultados para seguir testeando y no perder tanta velocidad
    # Cuando vayamos full mode, esta linea de abajo hay que comentarla o borrarla
    # Ahora, sabiendo esto, vamos a traer todos los resultados iterando y aumentando el offset en cada loop.
    # Recuerden que solo podemos traer de a tandas de a 50 resultados
    item_list = []
    categories_list=[]
    while r.status_code == 200 and offset <= maximum:
        url='https://api.mercadolibre.com/sites/MLA/search?category={}&limit=50&offset={}#json'.format(categ,offset)
        r = get_response(url)
        data = r.json()
        length = len(data['results'])
        for i in range(length):
            item_id = data['results'][i]['id']
            item_list.append(item_id)
            categories_list.append(categ)
        offset += 50
    return item_list,categories_list

In [6]:
len(categorias_full)

32

In [7]:
indices= [23, 25, 30]
categorias = [categorias_full[index] for index in indices]
categorias

[{'id': 'MLA3937', 'name': 'Joyas y Relojes'},
 {'id': 'MLA3025', 'name': 'Libros, Revistas y Comics'},
 {'id': 'MLA9304', 'name': 'Souvenirs, Cotillón y Fiestas'}]

In [8]:
maximum = 200

item_list=[]
categories_list=[]
for i in categorias:
    print("Buscamos items de la categoría {}".format(i['name']))
    temp_items,temp_categories=buscaritems(i['id'],0,maximum=maximum)
    item_list=item_list+temp_items
    categories_list=categories_list+temp_categories

Buscamos items de la categoría Joyas y Relojes
Encontramos 200 resultados para nuestra consulta
Buscamos items de la categoría Libros, Revistas y Comics
Encontramos 200 resultados para nuestra consulta
Buscamos items de la categoría Souvenirs, Cotillón y Fiestas
Encontramos 200 resultados para nuestra consulta


In [9]:
len(item_list), len(categories_list)

(750, 750)

In [10]:
categories_list

['MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'MLA3937',
 'ML

In [11]:
len(set(item_list))

1239

In [11]:
print('https://api.mercadolibre.com/items/{}'.format(item_list[0]))

https://api.mercadolibre.com/items/MLA853209990


In [12]:
item=get_response('https://api.mercadolibre.com/items/{}'.format(item_list[0]))

In [13]:
item.json()['pictures'][0]['secure_url']

'https://http2.mlstatic.com/D_729758-MLA46881046784_072021-O.jpg'

In [14]:
urllib.request.urlretrieve(item.json()['pictures'][0]['secure_url'], "imagen01")
img = Image.open("imagen01")
img.show()

In [15]:
item.json()

{'id': 'MLA853209990',
 'site_id': 'MLA',
 'title': 'Reloj Inteligente Pulsera - Smart Band Tedge',
 'subtitle': None,
 'seller_id': 417972986,
 'category_id': 'MLA352679',
 'official_store_id': 1824,
 'price': 1745,
 'base_price': 1745,
 'original_price': 2499,
 'currency_id': 'ARS',
 'initial_quantity': 7649,
 'available_quantity': 500,
 'sold_quantity': 5000,
 'sale_terms': [{'id': 'WARRANTY_TIME',
   'name': 'Tiempo de garantía',
   'value_id': None,
   'value_name': '6 meses',
   'value_struct': {'number': 6, 'unit': 'meses'},
   'values': [{'id': None,
     'name': '6 meses',
     'struct': {'number': 6, 'unit': 'meses'}}]},
  {'id': 'WARRANTY_TYPE',
   'name': 'Tipo de garantía',
   'value_id': '2230280',
   'value_name': 'Garantía del vendedor',
   'value_struct': None,
   'values': [{'id': '2230280',
     'name': 'Garantía del vendedor',
     'struct': None}]}],
 'buying_mode': 'buy_it_now',
 'listing_type_id': 'gold_special',
 'start_time': '2020-05-04T15:44:07.000Z',
 'stop_

In [16]:
item=[]
for j,i in enumerate(item_list):
    url='https://api.mercadolibre.com/items/{}'.format(i)
    item.append(get_response(url).json())
    print("Porcentaje de completitud: {:0.2%}".format((j+1)/len(item_list)),end='\r')

In [17]:
len(item)

750

In [18]:
def crearcarpeta(carpeta):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)

In [19]:
crearcarpeta('imagenes')

In [21]:
for z,i in enumerate(item):
    for w,j in enumerate(i['pictures']):
        crearcarpeta("./imagenes/{}".format(categories_list[z]))
        urllib.request.urlretrieve(j['url'],"./imagenes/{}/categ-{}_categl7-{}_itemid-{}_v{:02}.jpg".format(categories_list[z],categories_list[z],i['category_id'],i['id'],w))
    print("Porcentaje de completitud: {:0.2%}".format((z+1)/len(item)),end='\r')